# U-Net Model for RV Segmentation

## Training U-Net CNN

In [1]:
import sys
sys.path.append("..")
import os, re, glob
import numpy as np
import matplotlib as mplt
%matplotlib inline

from src import data,unet

Using TensorFlow backend.


In [2]:
train_dir = "/Users/aksharkumar/Documents/mlndCapstone/trainingdata/TrainingSet"


train_images=[]
train_inner_masks=[]
train_outer_masks = []

patient_directories = sorted(glob.glob(os.path.join(train_dir, "patient*")))

for patient_dir in patient_directories:
    imgdata = data.ImageData(patient_dir)
    train_images += imgdata.labeled_images
    train_inner_masks += imgdata.endo_masks.values()
    train_outer_masks += imgdata.epi_masks.values()

print("There are %d total training images." % len(train_images))
print("There are %d total inner masks." % len(train_inner_masks))
print("There are %d total outer masks." % len(train_outer_masks))


There are 243 total training images.
There are 243 total inner masks.
There are 243 total outer masks.


In [8]:
unet_conv = unet.UNet()

model = unet_conv.get_unet(height=216,width=256,channels=1,features=32,steps=3,padding='valid')

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 216, 256, 1)   0                                            
____________________________________________________________________________________________________
conv2d_35 (Conv2D)               (None, 214, 254, 32)  320         input_3[0][0]                    
____________________________________________________________________________________________________
conv2d_36 (Conv2D)               (None, 212, 252, 32)  9248        conv2d_35[0][0]                  
____________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)   (None, 106, 126, 32)  0           conv2d_36[0][0]                  
___________________________________________________________________________________________

In [6]:
from keras.optimizers import Adam
from keras import backend as K

def dice_coef(y_true, y_pred):
        flat_y_true = K.flatten(y_true)
        flat_y_preds = K.flatten(y_pred)
        intersection = K.sum(flat_y_true*flat_y_preds)
        return (2. * intersection + 1.) / (K.sum(flat_y_true)+K.sum(flat_y_preds))


model.compile(optimizer=Adam(lr=1e-5),loss='categorical_crossentropy',metrics=[dice_coef])

In [ ]:
#TODO: 1. split data into training and validation set
#      2. Augment the data
#      3. Train model
from keras.preprocessing.images import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='saved_models/weights.hdf5', verbose=1, save_best_only=True)

epochs = 10 

